In [3]:
import pandas as pd
import pycountry
import re

#Combine both Excel files
file1 = 'gov_docs_rand_200_2025_02_10.csv'
file2 = 'Random200Govdocs_Set2.csv'

# Load both csv files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Combine them and drop duplicates based on 'company_id'
df = pd.concat([df1, df2], ignore_index=True)
df.drop_duplicates(subset='company_id', inplace=True)

def clean_text(text):
    # Remove characters that are not allowed in Excel
    return re.sub(r'[^\x20-\x7E]', '', text)  # Keeps only printable characters

# Apply to the dataframe
df['extracted_text'] = df['extracted_text'].apply(clean_text)

In [5]:
##Country Names
# Get the names of all countries from pycountry
countries_list = [country.name for country in pycountry.countries]

# Add custom mappings 
custom_country_map = {
    'British': 'United Kingdom',
    'UK': 'United Kingdom',
    'USA': 'United States',
    'UAE': 'United Arab Emirates',
    'Congo': None,  # We'll handle this separately
}

# Add specific Congo-related entries
extra_country_terms = {
    'Democratic Republic of the Congo': 'Democratic Republic of the Congo',
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Republic of the Congo': 'Republic of the Congo'
}

# Combine everything into a search list
search_terms = countries_list + list(custom_country_map.keys()) + list(extra_country_terms.keys())

def find_country_in_text(text):
    if not isinstance(text, str):
        return None
    
    found_countries = set()

    for term in search_terms:
        if re.search(r'\b' + re.escape(term) + r'\b', text, re.IGNORECASE):
            # Handle custom mappings first
            if term in custom_country_map:
                mapped = custom_country_map[term]
                if mapped:  # Skip if None
                    found_countries.add(mapped)
            elif term in extra_country_terms:
                found_countries.add(extra_country_terms[term])
            else:
                found_countries.add(term)

    # Special case: if "Congo" is found and neither full name is, you can optionally add both
    if "Congo" in text and not any(c in found_countries for c in ['Democratic Republic of the Congo', 'Republic of the Congo']):
        found_countries.add("Congo (unspecified)")

    return list(found_countries) if found_countries else None

# Apply the function to the DataFrame
df['countries_from_text'] = df['extracted_text'].apply(find_country_in_text)

# Print results
print(df[['extracted_text', 'countries_from_text']])

# Save to Excel
df.to_excel('combined_countries.xlsx', index=False)

                                        extracted_text  \
0    Supply Chain Code of Conduct | Urban Logistics...   
1    Position Statement on Human Rights Abbott beli...   
2    3 2019 ACCELERATING PROGRESS TOWARD A SUSTAINA...   
3    GRI Sustainability Reporting Standards Skip to...   
4    Building foundations for a sustainable future ...   
..                                                 ...   
295  Supply Chain Transparency - Phillips 66 Skip t...   
297  Human Rights & Our Supply Chain | Marks & Spen...   
307  ExxonMobil statement on labor and the workplac...   
317  Our success is judged not only by commercial p...   
327  1 STATEMENT ON HUMAN RIGHTS Henry Schein, Inc....   

                                   countries_from_text  
0                                     [United Kingdom]  
1                                                 None  
2    [Mexico, Italy, United Kingdom, Australia, Chi...  
3                                              [Aruba]  
4                 

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load Excel file
file_path = "combined_countries.xlsx"  # Update this if needed
df = pd.read_excel(file_path)

# Prepare columns
df['sectors'] = df['sectors'].fillna('').str.split(';')
df['sub_industries'] = df['sub_industries'].fillna('').str.split(';')
df['countries_from_text'] = df['countries_from_text'].dropna().apply(eval)

# Explode to long format
df_exploded = df.explode('countries_from_text').explode('sectors').explode('sub_industries')
df_exploded = df_exploded.dropna(subset=['countries_from_text', 'sectors', 'sub_industries'])

# Create output folder
output_dir = "graphs2"
os.makedirs(output_dir, exist_ok=True)

# Loop through each sector and sub-industry
for sector in df_exploded['sectors'].unique():
    sector_df = df_exploded[df_exploded['sectors'] == sector]

    for sub in sector_df['sub_industries'].unique():
        sub_df = sector_df[sector_df['sub_industries'] == sub]

        # Get top 10 countries for this sub-industry
        top_countries = sub_df['countries_from_text'].value_counts().nlargest(10).index
        filtered_df = sub_df[sub_df['countries_from_text'].isin(top_countries)]

        # Count and plot
        count_data = (
            filtered_df.groupby('countries_from_text')
            .size()
            .reset_index(name='count')
            .sort_values(by='count', ascending=False)
        )

        plt.figure(figsize=(10, 6))
        sns.barplot(
            data=count_data,
            x='countries_from_text', y='count',
            order=count_data['countries_from_text']
        )
        plt.title(f"Top 10 Country Mentions\nSector: {sector} | Sub-Industry: {sub}", fontsize=12)
        plt.xlabel("Country")
        plt.ylabel("Mentions")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Save chart
        filename = f"{sector}_{sub}".replace('/', '_').replace('\\', '_').replace(' ', '_')
        filepath = os.path.join(output_dir, f"{filename}.png")
        plt.savefig(filepath)
        plt.close()
